<h1 align="center">---- DataFrames ----</h1>

----

<b>0) Packages</b>

In [1]:
using Pkg
#Pkg.add("CSV")   # reads/writes csv files
#Pkg.add("Arrow") # reads/writes arrow files

    Updating registry at `C:\Users\gabri\.julia\registries\General.toml`
   Resolving package versions...
   Installed FilePathsBase ─── v0.9.20
   Installed WeakRefStrings ── v1.4.2
   Installed WorkerUtilities ─ v1.6.1
   Installed CSV ───────────── v0.10.9
    Updating `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  [336ed68f] + CSV v0.10.9
    Updating `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
  [336ed68f] + CSV v0.10.9
  [48062228] + FilePathsBase v0.9.20
  [ea10d353] + WeakRefStrings v1.4.2
  [76eceee3] + WorkerUtilities v1.6.1
Precompiling project...
  ✓ WorkerUtilities
  ✓ WeakRefStrings
  ✓ FilePathsBase
  ✓ CSV
  4 dependencies successfully precompiled in 15 seconds. 185 already precompiled. 2 skipped during auto due to previous errors.
   Resolving package versions...
   Installed CEnum ─────── v0.4.2
   Installed Lz4_jll ───── v1.9.3+0
   Installed CodecLz4 ──── v0.4.0
   Installed Mocking ───── v0.7.5
   Installed ArrowTypes ── v2.0.2
   Installed Bi

In [1]:
using DataFrames
using CSV
using Arrow
using Statistics

----

<b>1) Loading Dataset</b>

In [2]:
# Downloading the text file
download(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original"
    , "auto.txt"
)

"auto.txt"

In [3]:
# Reading line per line
readlines("auto.txt")

406-element Vector{String}:
 "18.0   8.   307.0      130.0   " ⋯ 25 bytes ⋯ " 1.\t\"chevrolet chevelle malibu\""
 "15.0   8.   350.0      165.0   " ⋯ 17 bytes ⋯ "5   70.  1.\t\"buick skylark 320\""
 "18.0   8.   318.0      150.0   " ⋯ 18 bytes ⋯ "   70.  1.\t\"plymouth satellite\""
 "16.0   8.   304.0      150.0      3433.      12.0   70.  1.\t\"amc rebel sst\""
 "17.0   8.   302.0      140.0      3449.      10.5   70.  1.\t\"ford torino\""
 "15.0   8.   429.0      198.0      4341.      10.0   70.  1.\t\"ford galaxie 500\""
 "14.0   8.   454.0      220.0      4354.       9.0   70.  1.\t\"chevrolet impala\""
 "14.0   8.   440.0      215.0   " ⋯ 17 bytes ⋯ "5   70.  1.\t\"plymouth fury iii\""
 "14.0   8.   455.0      225.0      4425.      10.0   70.  1.\t\"pontiac catalina\""
 "15.0   8.   390.0      190.0   " ⋯ 18 bytes ⋯ "   70.  1.\t\"amc ambassador dpl\""
 "NA     4.   133.0      115.0   " ⋯ 20 bytes ⋯ " 70.  2.\t\"citroen ds-21 pallas\""
 "NA     8.   350.0      165.0   " ⋯ 32 bytes

In [4]:
# Storing the whole dataset in a variable
raw_data = read("auto.txt")

32149-element Vector{UInt8}:
 0x31
 0x38
 0x2e
 0x30
 0x20
 0x20
 0x20
 0x38
 0x2e
 0x20
 0x20
 0x20
 0x33
    ⋮
 0x63
 0x68
 0x65
 0x76
 0x79
 0x20
 0x73
 0x2d
 0x31
 0x30
 0x22
 0x0a

In [5]:
# Oops, do not forget to convert it to string in order to don't
# get an hexadecimal array
raw_data = read("auto.txt", String)

"18.0   8.   307.0      130.0      3504.      12.0   70.  1.\t\"chevrolet chevelle malibu\"\n15.0   8.   350.0      165.0      3693.      11.5   70.  1.\t\"buick skylark 320\"\n18.0   8.   318.0      150.0      3436.      11.0   70.  1.\t\"plymouth satellite\"\n16.0   8.   304.0    " ⋯ 31610 bytes ⋯ "00      2130.      24.6   82.  2.\t\"vw pickup\"\n32.0   4.   135.0      84.00      2295.      11.6   82.  1.\t\"dodge rampage\"\n28.0   4.   120.0      79.00      2625.      18.6   82.  1.\t\"ford ranger\"\n31.0   4.   119.0      82.00      2720.      19.4   82.  1.\t\"chevy s-10\"\n"

In [6]:
# Oh, our dataset have a bunch of "\t" scapes. Let's replace it
# by an empty String
raw_data_with_no_tab = replace(raw_data, '\t'=>' ')

"18.0   8.   307.0      130.0      3504.      12.0   70.  1. \"chevrolet chevelle malibu\"\n15.0   8.   350.0      165.0      3693.      11.5   70.  1. \"buick skylark 320\"\n18.0   8.   318.0      150.0      3436.      11.0   70.  1. \"plymouth satellite\"\n16.0   8.   304.0    " ⋯ 31610 bytes ⋯ "00      2130.      24.6   82.  2. \"vw pickup\"\n32.0   4.   135.0      84.00      2295.      11.6   82.  1. \"dodge rampage\"\n28.0   4.   120.0      79.00      2625.      18.6   82.  1. \"ford ranger\"\n31.0   4.   119.0      82.00      2720.      19.4   82.  1. \"chevy s-10\"\n"

In [7]:
# Now, let's create an "IOBuffer" variable to simulate a CSV File
# with this data
io = IOBuffer(raw_data_with_no_tab)

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=32149, maxsize=Inf, ptr=1, mark=-1)

In [8]:
# And, convert it to a DataFrame
df = CSV.File(
    io
    , delim=' '
    , ignorerepeated=true
    , header=[:mpg, :cylinders, :displacement, :horsepower,
              :weight, :acceleration, :year, :origin, :name]
    , missingstring="NA"
) |> DataFrame

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


----

<b>2) Another Way to do it</b>

In [9]:
# Reading dataset
df = CSV.File("auto.txt", header=[:metrics, :name]) |> DataFrame

Row,metrics,name
,String,String
1,18.0 8. 307.0 130.0 3504. 12.0 70. 1.,chevrolet chevelle malibu
2,15.0 8. 350.0 165.0 3693. 11.5 70. 1.,buick skylark 320
3,18.0 8. 318.0 150.0 3436. 11.0 70. 1.,plymouth satellite
4,16.0 8. 304.0 150.0 3433. 12.0 70. 1.,amc rebel sst
5,17.0 8. 302.0 140.0 3449. 10.5 70. 1.,ford torino
6,15.0 8. 429.0 198.0 4341. 10.0 70. 1.,ford galaxie 500
7,14.0 8. 454.0 220.0 4354. 9.0 70. 1.,chevrolet impala
8,14.0 8. 440.0 215.0 4312. 8.5 70. 1.,plymouth fury iii
9,14.0 8. 455.0 225.0 4425. 10.0 70. 1.,pontiac catalina


In [10]:
# Splitting metrics by spaces
metrics_array = split.(df.metrics)

406-element Vector{Vector{SubString{String}}}:
 ["18.0", "8.", "307.0", "130.0", "3504.", "12.0", "70.", "1."]
 ["15.0", "8.", "350.0", "165.0", "3693.", "11.5", "70.", "1."]
 ["18.0", "8.", "318.0", "150.0", "3436.", "11.0", "70.", "1."]
 ["16.0", "8.", "304.0", "150.0", "3433.", "12.0", "70.", "1."]
 ["17.0", "8.", "302.0", "140.0", "3449.", "10.5", "70.", "1."]
 ["15.0", "8.", "429.0", "198.0", "4341.", "10.0", "70.", "1."]
 ["14.0", "8.", "454.0", "220.0", "4354.", "9.0", "70.", "1."]
 ["14.0", "8.", "440.0", "215.0", "4312.", "8.5", "70.", "1."]
 ["14.0", "8.", "455.0", "225.0", "4425.", "10.0", "70.", "1."]
 ["15.0", "8.", "390.0", "190.0", "3850.", "8.5", "70.", "1."]
 ["NA", "4.", "133.0", "115.0", "3090.", "17.5", "70.", "2."]
 ["NA", "8.", "350.0", "165.0", "4142.", "11.5", "70.", "1."]
 ["NA", "8.", "351.0", "153.0", "4034.", "11.0", "70.", "1."]
 ⋮
 ["25.0", "6.", "181.0", "110.0", "2945.", "16.4", "82.", "1."]
 ["38.0", "6.", "262.0", "85.00", "3015.", "17.0", "82.", "1."]

In [11]:
# Creating a new dataframe with just the headers
new_df = DataFrame(
    [col => Float64[] for 
    col in [:mpg, :cylinders, :displacement, :horsepower,
            :weight, :acceleration, :year, :origin]]
)

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [12]:
# Enabling missing values to "mpg" and "horsepower" features
allowmissing!(new_df, [:mpg, :horsepower])

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64


In [13]:
# Setting the metrics to it
for row in metrics_array
    push!(new_df, [v == "NA" ? missing : parse(Float64, v) for v in row])
end

new_df

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0


In [14]:
# Adding the "name" feature
new_df.name = df.name

406-element Vector{String}:
 "chevrolet chevelle malibu"
 "buick skylark 320"
 "plymouth satellite"
 "amc rebel sst"
 "ford torino"
 "ford galaxie 500"
 "chevrolet impala"
 "plymouth fury iii"
 "pontiac catalina"
 "amc ambassador dpl"
 "citroen ds-21 pallas"
 "chevrolet chevelle concours (sw)"
 "ford torino (sw)"
 ⋮
 "buick century limited"
 "oldsmobile cutlass ciera (diesel)"
 "chrysler lebaron medallion"
 "ford granada l"
 "toyota celica gt"
 "dodge charger 2.2"
 "chevrolet camaro"
 "ford mustang gl"
 "vw pickup"
 "dodge rampage"
 "ford ranger"
 "chevy s-10"

In [15]:
# Final Result
new_df

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


----

<b>3) The Way I Would Do</b>

In [16]:
# Reading the dataset
df = CSV.File("auto.txt", header=[:metrics, :name]) |> DataFrame

Row,metrics,name
,String,String
1,18.0 8. 307.0 130.0 3504. 12.0 70. 1.,chevrolet chevelle malibu
2,15.0 8. 350.0 165.0 3693. 11.5 70. 1.,buick skylark 320
3,18.0 8. 318.0 150.0 3436. 11.0 70. 1.,plymouth satellite
4,16.0 8. 304.0 150.0 3433. 12.0 70. 1.,amc rebel sst
5,17.0 8. 302.0 140.0 3449. 10.5 70. 1.,ford torino
6,15.0 8. 429.0 198.0 4341. 10.0 70. 1.,ford galaxie 500
7,14.0 8. 454.0 220.0 4354. 9.0 70. 1.,chevrolet impala
8,14.0 8. 440.0 215.0 4312. 8.5 70. 1.,plymouth fury iii
9,14.0 8. 455.0 225.0 4425. 10.0 70. 1.,pontiac catalina


In [17]:
# Transforming the datas
new_df = select(
    df
    , :metrics =>
    ByRow(x -> something.(tryparse.(Float64, split(x)), missing)) =>
    [:mpg, :cylinders, :displacement, :horsepower, :weight, :acceleration, :year, :origin]
    , :name
)

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


----

<b>4) Operations</b>

In [18]:
# Counting Missing Values
count(ismissing, Matrix(new_df))

14

In [19]:
# Filtering Rows (that has missing values)
filter(row -> any(ismissing, row), new_df)

Row,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,missing,4.0,133.0,115.0,3090.0,17.5,70.0,2.0,citroen ds-21 pallas
2,missing,8.0,350.0,165.0,4142.0,11.5,70.0,1.0,chevrolet chevelle concours (sw)
3,missing,8.0,351.0,153.0,4034.0,11.0,70.0,1.0,ford torino (sw)
4,missing,8.0,383.0,175.0,4166.0,10.5,70.0,1.0,plymouth satellite (sw)
5,missing,8.0,360.0,175.0,3850.0,11.0,70.0,1.0,amc rebel sst (sw)
6,missing,8.0,302.0,140.0,3353.0,8.0,70.0,1.0,ford mustang boss 302
7,25.0,4.0,98.0,missing,2046.0,19.0,71.0,1.0,ford pinto
8,missing,4.0,97.0,48.0,1978.0,20.0,71.0,2.0,volkswagen super beetle 117
9,21.0,6.0,200.0,missing,2875.0,17.0,74.0,1.0,ford maverick


In [20]:
# Shape (rows, columns)
size(df)

(406, 2)

----

<b>5) Saving the DataSet in File</b>

In [64]:
# CSV Format
CSV.write("auto2.csv", new_df)

"auto2.csv"

In [66]:
# Arrow Format
Arrow.write("auto2.arrow", new_df)

"auto2.arrow"

In [67]:
# Reaind the CSV File
readlines("auto2.csv")

407-element Vector{String}:
 "mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name"
 "18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu"
 "15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320"
 "18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite"
 "16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst"
 "17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino"
 "15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500"
 "14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala"
 "14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii"
 "14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina"
 "15.0,8.0,390.0,190.0,3850.0,8.5,70.0,1.0,amc ambassador dpl"
 ",4.0,133.0,115.0,3090.0,17.5,70.0,2.0,citroen ds-21 pallas"
 ",8.0,350.0,165.0,4142.0,11.5,70.0,1.0,chevrolet chevelle concours (sw)"
 ⋮
 "25.0,6.0,181.0,110.0,2945.0,16.4,82.0,1.0,buick century limited"
 "38.0,6.0,262.0,85.0,3015.0,17.0,82.0,1.0,oldsmobile cutlass ci

In [68]:
# Reading Arrow File
readlines("auto2.arrow")

30-element Vector{String}:
 "ARROW1\0\0\xff\xff\xff\xff \x02\0\0\x10\0\0\0\0\0"
 "\0\f\0"
 "\0\b\0\x04\0"
 "\0\0\0\x10\0\0\0\x01\0\x04\0\b\0\b\0\0\0\x04\0\b\0\0\0\x04\0\0\0\t\0\0\0" ⋯ 266 bytes ⋯ "\xff\xff\x10\0\0\0\x10\0\0\0\x03\0\0\x01\x10\0\0\0\0\0\0\0J\xff\xff\xff\0\0\x02\0"
 "\0\0\0horsepower\0\0\xcc\xff\xff\xff\x10\0\0\0\x10\0\0\0\0\0\x03\0" ⋯ 188 bytes ⋯ "\x14\0\x12\0\f\0\x04\0\f\0\0\0\0\x86\0\0\0\0\0\0\x14\0\0\0\0\0\x03\0\x04\0"
 "\0\x18\0\f\0\b\0\x04\0"
 "\0\0\0\x14\0\0\0H\x01\0\0\x96\x01\0\0\0\0\0\0\0\0\0\0\x13\0\0\0\0\0\0" ⋯ 15236 bytes ⋯ "@\0\0\0\0\0\x84\x9c@\0\0\0\0\0\x18\x9f@\0\0\0\0\0֠@\0\0\0\0\0"
 "\xa4@\0\0\0\0\0\x9c\xa8@\0\0\0\0\0Ȯ@\0\0\0\0\0\x8c\xa9@\0\0\0\0\0\xe4\xa6" ⋯ 398 bytes ⋯ "H\xac@\0\0\0\0\0\xa4\xaa@\0\0\0\0\0ª@\0\0\0\0\0\xea\xaa@\0\0\0\0\0"
 "\xa9@\0\0\0\0\0\xe0\xaf@\0\0\0\0\0֠@\0\0\0\0\0\0\xa4@\0\0\0\0\0\xf8" ⋯ 11009 bytes ⋯ "\0\xa9\x03\0\0\xbb\x03\0\0\xc4\x03\0\0\xcf\x03\0\0\xd8\x03\0\0\xeb\x03\0\0\xf6\x03\0\0"
 "\x04\0\0\x1a\x04\0\0/\x04\0\0A\x04\0\0R\x0